<a href="https://colab.research.google.com/github/Prasanthsai0987/Quantum-Machine-Learning-Tasks-/blob/main/QML%20%20(Task%20-%204)%20%20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

============================================================
TASK 4: COMMUTATION RELATIONS AND EULER ANGLES
============================================================

AIM:
-----
To verify the commutation and anti-commutation relations
of the Pauli matrices and to perform the Z–Y–Z Euler
angle decomposition of single-qubit unitary operators.

------------------------------------------------------------
ALGORITHM:
------------------------------------------------------------

Step 1: Import required libraries (NumPy, SymPy, cmath).

Step 2: Define Pauli matrices σx, σy, σz.

Step 3: Define functions:
          comm(A, B)  → Compute commutator [A, B]
          anti(A, B)  → Compute anti-commutator {A, B}

Step 4: Verify:
          [σx, σy] = 2iσz,
          [σy, σz] = 2iσx,
          [σz, σx] = 2iσy
          and
          {σi, σj} = 2δij I

Step 5: Define helper functions for rotations:
          Rz(θ), Ry(θ), and Rx(θ)

Step 6: Implement Euler decomposition function:
          decompose_zyz(U) →
             - Verify U is unitary
             - Extract global phase φ
             - Compute α, β, γ such that
               U = e^{iφ} Rz(α) Ry(β) Rz(γ)

Step 7: Test decomposition for common gates:
          Rx(π/3), Ry(π/4), Rz(π/2), H, S, T

Step 8: (Optional) Validate using Cirq unitary matrices.


------------------------------------------------------------
MATHEMATICAL MODEL:
------------------------------------------------------------

1. Pauli Matrices:

σx = [[0, 1],
      [1, 0]]

σy = [[0, -i],
      [i,  0]]

σz = [[1,  0],
      [0, -1]]

------------------------------------------------------------
2. Commutation and Anti-Commutation Relations:

Commutator:       [A, B] = A·B − B·A  
Anti-Commutator:  {A, B} = A·B + B·A

For Pauli matrices:
   [σx, σy] = 2i σz
   [σy, σz] = 2i σx
   [σz, σx] = 2i σy

   {σi, σj} = 2δij I    (where δij is the Kronecker delta)

These relations confirm that the Pauli matrices form a
Lie algebra basis for SU(2).

------------------------------------------------------------
3. Euler Angle Decomposition (Z–Y–Z Form):

Any single-qubit unitary U ∈ SU(2) can be expressed as:

U = e^{iφ} Rz(α) Ry(β) Rz(γ)

where:
   Rz(θ) = [[e^{-iθ/2}, 0],
             [0, e^{iθ/2}]]

   Ry(θ) = [[cos(θ/2), -sin(θ/2)],
             [sin(θ/2),  cos(θ/2)]]

and φ, α, β, γ are the **Euler angles**.

------------------------------------------------------------
4. Extraction of Euler Angles:

Given a unitary U:
   (a) Remove global phase → det(U₀) = 1  
   (b) Compute:
       a = U₀₀,  b = U₀₁
       β = 2·arccos(|a|)
       If sin(β/2) ≠ 0:
           α = arg(a) + arg(b)
           γ = arg(a) − arg(b)
       Else:
           α = 2·arg(a),  γ = 0
   (c) Global phase: φ = (arg(det(U)))/2

In [ ]:
import numpy as np
import cmath
import sympy as sp

print("\n" + "="*50)
print("TASK 4: COMMUTATION RELATIONS AND EULER ANGLES")
print("="*50)

# --- Part 1: Verify Pauli commutation & anti-commutation with SymPy ---
I = sp.eye(2)
sx = sp.Matrix([[0, 1], [1, 0]])
sy = sp.Matrix([[0, -sp.I], [sp.I, 0]])
sz = sp.Matrix([[1, 0], [0, -1]])
paulis = {'X': sx, 'Y': sy, 'Z': sz}

def comm(A, B):
    return sp.simplify(A * B - B * A)

def anti(A, B):
    return sp.simplify(A * B + B * A)

print("\n=== Commutation relations ===")
for (a, b, k) in [('X', 'Y', 'Z'), ('Y', 'Z', 'X'), ('Z', 'X', 'Y')]:
    lhs = comm(paulis[a], paulis[b])
    rhs = 2 * sp.I * paulis[k]
    print(f"[{a},{b}] =\n{lhs}\nExpected:\n{rhs}\n")

print("\n=== Anti-commutation relations ===")
for i in ['X', 'Y', 'Z']:
    for j in ['X', 'Y', 'Z']:
        lhs = anti(paulis[i], paulis[j])
        rhs = 2 * (1 if i == j else 0) * I
        print(f"{{{i},{j}}} =\n{lhs} Expected:\n{rhs}\n")

# --- Part 2: Z–Y–Z Euler decomposition ---
def is_unitary(U, tol=1e-8):
    return np.allclose(U.conj().T @ U, np.eye(2), atol=tol)

def decompose_zyz(U, tol=1e-8):
    """
    Return (phi, alpha, beta, gamma) such that
    U = e^{i phi} Rz(alpha) Ry(beta) Rz(gamma)
    """
    U = np.array(U, dtype=complex)
    if not is_unitary(U):
        raise ValueError("Matrix is not unitary.")

    detU = np.linalg.det(U)
    phi = cmath.phase(detU) / 2
    U0 = U * np.exp(-1j * phi)
    detU0 = np.linalg.det(U0)
    U0 = U0 / np.sqrt(detU0)

    a = U0[0, 0]
    b = U0[0, 1]

    beta = 2 * np.arccos(min(1.0, max(0.0, abs(a))))

    if np.isclose(np.sin(beta / 2), 0, atol=tol):
        alpha = 2 * (-cmath.phase(a))
        gamma = 0.0
    else:
        phi1 = -cmath.phase(a)
        phi2 = -cmath.phase(-b)
        alpha = phi1 + phi2
        gamma = phi1 - phi2

    return float(phi), float(alpha), float(beta), float(gamma)

def Rz(theta):
    return np.array([
        [np.exp(-1j * theta / 2), 0],
        [0, np.exp(1j * theta / 2)]
    ], dtype=complex)

def Ry(theta):
    return np.array([
        [np.cos(theta / 2), -np.sin(theta / 2)],
        [np.sin(theta / 2), np.cos(theta / 2)]
    ], dtype=complex)

def reconstruct(phi, alpha, beta, gamma):
    return np.exp(1j * phi) * (Rz(alpha) @ Ry(beta) @ Rz(gamma))

# --- Part 3: Test examples ---
def Rx(theta):
    return np.cos(theta / 2) * np.eye(2) - 1j * np.sin(theta / 2) * np.array([[0, 1], [1, 0]])

examples = {
    "Rx(pi/3)": Rx(np.pi / 3),
    "Ry(pi/4)": Ry(np.pi / 4),
    "Rz(pi/2)": Rz(np.pi / 2),
    "H": (1 / np.sqrt(2)) * np.array([[1, 1], [1, -1]], dtype=complex),
    "S": np.array([[1, 0], [0, 1j]], dtype=complex),
    "T": np.array([[1, 0], [0, np.exp(1j * np.pi / 4)]], dtype=complex),
}

print("\n=== Z–Y–Z Euler Decomposition ===")
for name, U in examples.items():
    phi, alpha, beta, gamma = decompose_zyz(U)
    print(f"{name}:\n φ = {phi:.6f}, α = {alpha:.6f}, β = {beta:.6f}, γ = {gamma:.6f}\n")

# --- Optional: Cirq-based test ---
try:
    import cirq
    print("\nCirq example decomposition for H gate:")
    q = cirq.LineQubit(0)
    H_op = cirq.H(q)
    U = cirq.unitary(H_op)
    phi, alpha, beta, gamma = decompose_zyz(U)
    print(f"Cirq H:\n φ = {phi:.6f}, α = {alpha:.6f}, β = {beta:.6f}, γ = {gamma:.6f}\n")
except ImportError:
    print("\nCirq not installed. Skipping Cirq examples.")



TASK 4: COMMUTATION RELATIONS AND EULER ANGLES

=== Commutation relations ===
[X,Y] =
Matrix([[2*I, 0], [0, -2*I]])
Expected:
Matrix([[2*I, 0], [0, -2*I]])

[Y,Z] =
Matrix([[0, 2*I], [2*I, 0]])
Expected:
Matrix([[0, 2*I], [2*I, 0]])

[Z,X] =
Matrix([[0, 2], [-2, 0]])
Expected:
Matrix([[0, 2], [-2, 0]])


=== Anti-commutation relations ===
{X,X} =
Matrix([[2, 0], [0, 2]]) Expected:
Matrix([[2, 0], [0, 2]])

{X,Y} =
Matrix([[0, 0], [0, 0]]) Expected:
Matrix([[0, 0], [0, 0]])

{X,Z} =
Matrix([[0, 0], [0, 0]]) Expected:
Matrix([[0, 0], [0, 0]])

{Y,X} =
Matrix([[0, 0], [0, 0]]) Expected:
Matrix([[0, 0], [0, 0]])

{Y,Y} =
Matrix([[2, 0], [0, 2]]) Expected:
Matrix([[2, 0], [0, 2]])

{Y,Z} =
Matrix([[0, 0], [0, 0]]) Expected:
Matrix([[0, 0], [0, 0]])

{Z,X} =
Matrix([[0, 0], [0, 0]]) Expected:
Matrix([[0, 0], [0, 0]])

{Z,Y} =
Matrix([[0, 0], [0, 0]]) Expected:
Matrix([[0, 0], [0, 0]])

{Z,Z} =
Matrix([[2, 0], [0, 2]]) Expected:
Matrix([[2, 0], [0, 2]])


=== Z–Y–Z Euler Decomposition ===
Rx